In [46]:
from h5Dataset import h5Dataset

In [47]:
import torch
from torch.utils.data import Dataset
from torch_geometric.data import Data 
from torch_geometric.loader import DataLoader 
from torch_geometric.utils import dense_to_sparse

import time
import h5py
import math
import sys
from pathlib import Path


In [58]:
dataset = h5Dataset("../data/test.h5")
loader = DataLoader(dataset, batch_size=1, num_workers=0)

start = time.perf_counter()

for i, batch in enumerate(loader):
    print(batch[0])

elapsed = time.perf_counter() - start
print(f"Total time: {elapsed:.3f}s")
print(f"Time per batch: {elapsed / len(loader):.6f}s")

DataBatch(x=[5, 7], edge_index=[2, 20], edge_attr=[20, 5], y=[3], batch=[5], ptr=[2])
DataBatch(x=[5, 7], edge_index=[2, 20], edge_attr=[20, 5], y=[3], batch=[5], ptr=[2])
DataBatch(x=[5, 7], edge_index=[2, 20], edge_attr=[20, 5], y=[3], batch=[5], ptr=[2])
DataBatch(x=[5, 7], edge_index=[2, 20], edge_attr=[20, 5], y=[3], batch=[5], ptr=[2])
DataBatch(x=[5, 7], edge_index=[2, 20], edge_attr=[20, 5], y=[3], batch=[5], ptr=[2])
Total time: 0.008s
Time per batch: 0.001526s
